In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
pd.options.display.float_format = '{:.0f}'.format

In [2]:
nurse_df = pd.read_csv('https://raw.githubusercontent.com/datadesk/california-coronavirus-data/master/cdph-skilled-nursing-totals.csv')
nurse_df.head(5)

,date,staff_active_cases,patients_active_cases,staff_confirmed_cases,patients_confirmed_cases,staff_deaths,patients_deaths,staff_deaths_note,source_url
0,2021-05-04,128,64,50722,61422,229,9050,NaN,https://www.cdph.ca.gov/Programs/CID/DCDC/Page...
1,2021-05-03,125,63,50715,61409,229,9050,NaN,https://www.cdph.ca.gov/Programs/CID/DCDC/Page...
2,2021-05-01,120,53,50692,61388,229,9048,NaN,https://www.cdph.ca.gov/Programs/CID/DCDC/Page...
3,2021-04-30,132,54,50687,61386,229,9048,NaN,https://www.cdph.ca.gov/Programs/CID/DCDC/Page...
4,2021-04-29,149,46,50678,61380,229,9047,NaN,https://www.cdph.ca.gov/Programs/CID/DCDC/Page...


In [3]:
nurse_df.drop(nurse_df.columns[[1, 2, 7, 8]], axis=1, inplace=True)
nurse_df['type'] = 'SN'
nurse_df

,date,staff_confirmed_cases,patients_confirmed_cases,staff_deaths,patients_deaths,type
0,2021-05-04,50722,61422,229,9050,SN
1,2021-05-03,50715,61409,229,9050,SN
2,2021-05-01,50692,61388,229,9048,SN
3,2021-04-30,50687,61386,229,9048,SN
4,2021-04-29,50678,61380,229,9047,SN
...,...,...,...,...,...,...
337,2020-04-29,2594,3923,19,618,SN
338,2020-04-28,2323,3515,1,578,SN
339,2020-04-25,2329,3441,1,545,SN
340,2020-04-24,2099,3141,1,486,SN


In [4]:
today = nurse_df[(nurse_df['date'] == '2021-05-04')]
today.set_index('date', inplace=True)
today

,staff_confirmed_cases,patients_confirmed_cases,staff_deaths,patients_deaths,type
date,,,,,
2021-05-04,50722,61422,229,9050,SN


In [5]:
senior_df = pd.read_csv('https://raw.githubusercontent.com/datadesk/california-coronavirus-data/master/cdph-adult-and-senior-care-totals.csv')

In [6]:
senior_df.drop(senior_df.columns[[3, 4]], axis=1, inplace=True)
senior_df['type'] = 'SC'
sc_today = senior_df[(senior_df['date'] == '2021-04-30')]
sc_today

,date,confirmed_cases,deaths,type
0,2021-04-30,42573,3739,SC


In [7]:
sc_today.set_index('date', inplace=True)
sc_today

,confirmed_cases,deaths,type
date,,,
2021-04-30,42573,3739,SC


In [8]:
latest_df = pd.merge(today, sc_today, left_on=['date'], right_on=['date'])
latest_df

,staff_confirmed_cases,patients_confirmed_cases,staff_deaths,patients_deaths,type_x,confirmed_cases,deaths,type_y
date,,,,,,,,


In [9]:
combined = today.merge(sc_today, how='outer', left_index=True, right_index=True)
combined

,staff_confirmed_cases,patients_confirmed_cases,staff_deaths,patients_deaths,type_x,confirmed_cases,deaths,type_y
date,,,,,,,,
2021-04-30,nan,nan,nan,nan,NaN,42573,3739,SC
2021-05-04,50722,61422,229,9050,SN,nan,nan,NaN


In [10]:
combined['patient_cases'] = combined["patients_confirmed_cases"] + combined["confirmed_cases"]
combined['patient_deaths'] = combined["patients_deaths"] + combined["deaths"]
combined

,staff_confirmed_cases,patients_confirmed_cases,staff_deaths,patients_deaths,type_x,confirmed_cases,deaths,type_y,patient_cases,patient_deaths
date,,,,,,,,,,
2021-04-30,nan,nan,nan,nan,NaN,42573,3739,SC,nan,nan
2021-05-04,50722,61422,229,9050,SN,nan,nan,NaN,nan,nan


In [11]:
combined2 = combined[['patient_cases', 'patient_deaths', 'staff_confirmed_cases', 'staff_deaths']]
combined2

,patient_cases,patient_deaths,staff_confirmed_cases,staff_deaths
date,,,,
2021-04-30,nan,nan,nan,nan
2021-05-04,nan,nan,50722,229
